In [1]:
import requests
import datetime
import time
import pandas as pd
import numpy as np

In [4]:
LOGIN_URL = 'https://yb01.88lard.com/api/v1/manager/login'
headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
            'accept-language': 'zh-TW'
}
payload = {
    'username': 'bbtorin',
    'password': 'qwe123',
}

time1=(datetime.datetime.now() - datetime.timedelta(hours=36)).strftime("%Y-%m-%d") 

In [3]:
#登入
session_requests = requests.session()
response = session_requests.request('PUT',url=LOGIN_URL, data=payload, headers=headers)
#反水
ttt = session_requests.get("https://yb01.88lard.com/api/v1/rebate/event/list?first_result=0&max_results=20&frequency=2&start_at="+str(delay_oneday)+"T00%3A00%3A00-04%3A00&end_at="+str(delay_oneday)+"T23%3A59%3A59-04%3A00" , headers=headers )
output_data=ttt.json()['ret']
df_rebet=pd.DataFrame(output_data)[['name','total']]
#總餘額有效投注損益
ttt = session_requests.get("https://yb01.88lard.com/api/v1/stats/daily_report?start_at="+str(delay_oneday)+"T00%3A00%3A00-04%3A00&end_at="+str(delay_oneday)+"T23%3A59%3A59-04%3A00" , headers=headers )
output_data=ttt.json()['ret']
df_total_data=pd.DataFrame(output_data).T[['valid_bet','payoff','total_balance']]

#測試帳號撈取
session_requests = requests.session()
response = session_requests.request('PUT',url=LOGIN_URL, data=payload, headers=headers)
ttt = session_requests.get("https://yb01.88lard.com/api/v1/player/list?level=581&enable=1&first_deposit=3&country_code=0&search=user&first_result=0&max_results=20&sort=id&order=desc&use_cache=true&fields=bankrupt&fields=blacklist&fields=cash&fields=enable&fields=id&fields=last_city_id&fields=last_country&fields=last_ip&fields=last_login&fields=last_online&fields=level&fields=locked&fields=parent&fields=tied&fields=username&fields=upper" , headers=headers )
output_data=ttt.json()['ret']
test_user=pd.DataFrame(output_data)[['username','enable']]

#計算有效投注人數
ttt = session_requests.get("https://yb01.88lard.com/api/v1/stats/agents/wager_report?canceled=0&start_at="+str(delay_oneday)+"T00%3A00%3A00-04%3A00&end_at="+str(delay_oneday)+"T23%3A59%3A59-04%3A00&timeOption=at&currency=&to_CNY=true&specify=0&=&=&=&parentOption=all&first_result=0&max_results=20" , headers=headers )
output_data=ttt.json()['ret']
parent_id=pd.DataFrame(output_data)[['parent_id','user_count']]

total_list=[]
list_data_all=[]
for i in range(parent_id.shape[0]) : 
    p_id=parent_id.iloc[i,0]
    u_count=parent_id.iloc[i,1]
    list_data_all=[]
    for j in range(0,int(u_count),1000): 
        ttt = session_requests.get("https://yb01.88lard.com/api/v1/stats/agent/"+str(p_id)+"/children/wager_report?start_at="+str(delay_oneday)+"T00%3A00%3A00-04%3A00&end_at="+str(delay_oneday)+"T23%3A59%3A59-04%3A00&canceled=0&first_result="+str(j)+"&max_results=1000&currency=&to_CNY=true" , headers=headers )
        output_data=ttt.json()['ret']
        list_data_all=list_data_all+output_data
    total_list=total_list+list_data_all
all_bet_user=pd.DataFrame(total_list)[['username','counts']]

formal_user_bet=pd.merge(all_bet_user,test_user,on = 'username',how = 'left')
user_bet_total=formal_user_bet[~(formal_user_bet['enable']==True)]

In [24]:
df_rebet

,name,total
0,2021-01-09 日返,193085.3452


In [25]:
round(float(df_rebet.iloc[0,1]),2)

193085.35

In [7]:
user_bet_total.shape[0]

2145

In [98]:
user_bet_total.to_csv('F:/Desktop/tableau_data/test_user.csv' ,encoding="utf_8_sig" )

In [108]:
user_bet_total

,username,counts,enable
0,aaaa234,169,NaN
1,l87984545,4212,NaN
2,e12316,42,NaN
3,qq335759136,87,NaN
4,wydycb,1,NaN
...,...,...,...
2998,z627098537,153,NaN
2999,yz19970520,5,NaN
3000,qazwsxpqp,202,NaN
3001,qing0513,66,NaN


In [66]:
session_requests = requests.session()
response = session_requests.request('PUT',url=LOGIN_URL, data=payload, headers=headers)
list_data_all=[]
for j in range(0,2000,500):
    ttt = session_requests.get("https://yb01.88lard.com/api/v1/withdraw/list?status_total=true&first_result="+str(j)+"&max_results=500&start_created_at=2021-01-05T00%3A00%3A00-04%3A00&end_created_at=2021-01-05T23%3A59%3A59-04%3A00&confirm=true&kind=1" , headers=headers )
    output_data=ttt.json()['ret']
    list_data_all=list_data_all+output_data
bank_user=pd.DataFrame(list_data_all)[['username','user_name','province','city','branch','amount','risk_status']]

In [67]:
bank_user.to_csv('F:/Desktop/tableau_data/bank_user.csv' ,encoding="utf_8_sig" )

In [99]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [109]:
scopes = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("F:/Desktop/stalwart-glider-298002-d11d931ddf61.json", scopes)
client = gspread.authorize(credentials)
sheet = client.open_by_key("1RykapcQop4jjZzRU6SbyY_FSF7fp3UxX3U1JvFH70oc").sheet1

In [5]:
#登入
session_requests = requests.session()
response = session_requests.request('PUT',url=LOGIN_URL, data=payload, headers=headers)

In [7]:
#會員
list_data_all=[]
k=0
for j in range(0,3000000,1000):
    ttt = session_requests.get("https://yb01.88lard.com/api/v1/player/list?enable=1&bankrupt=0&locked=0&first_result="+str(j)+"&max_results=1000&sort=id&order=desc&use_cache=true&fields=cash&fields=enable&fields=id&fields=last_login&fields=level&fields=username" , headers=headers )
    output_data=ttt.json()['ret']
    list_data_all=list_data_all+output_data
    if j%50000==0:
        #登入
        if k==0 :
            payload = {
                'username': 'bbtorin',
                'password': 'qwe123',
            }
            k=1
        if k==1 :
            payload = {
                'username': 'btorin',
                'password': 'qwe123',
            }
            k=0
        session_requests = requests.session()
        response = session_requests.request('PUT',url=LOGIN_URL, data=payload, headers=headers)
        print(j)

0
50000
100000


KeyboardInterrupt: 

In [76]:
df_total_data=pd.DataFrame(list_data_all) 
df_testdata=df_total_data['level'].astype(str).str.replace("{","").str.replace("}","").str.split(',',expand=True)
df_total_data['user_level']=df_testdata[1].str.replace(" 'name': ","").str.replace("'","")
df_balance=df_total_data['cash'].astype(str).str.replace("{","").str.replace("}","").str.split(',',expand=True)
df_total_data['user_balance']=df_balance[0].str.replace("'balance': ","").str.replace("'","")
df_data_all=df_total_data[['username','last_login','user_level','user_balance']]

In [ ]:
df_data_all.to_csv('F:/Desktop/user_data/user_list.csv' ,encoding="utf_8_sig" )

In [2]:
yabo_user = pd.read_csv("F:/Desktop/daily_register_YABO.csv" , encoding = "utf-8")
sg_user = pd.read_csv("F:/Desktop/daily_register_SG.csv" , encoding = "utf-8")

In [27]:
yaboip=np.unique(yabo_user['注册IP'])
sgip=np.unique(sg_user['注册IP'])

In [31]:
len(list(set(yaboip).intersection(set(sgip))))

370

In [4]:
test_user=pd.merge(yabo_user,sg_user,on = '注册IP',how = 'left')

In [10]:
test_user.columns

Index(['序_x', '会员帐号_x', '真实姓名_x', '直属好友_x', '代理帐号_x', '注册IP', '注册时间_x',
       '注册国家_x', '注册城市_x', '主帐户余额_x', '损益_x', '会员层级_x', '启/停用_x', '冻结_x',
       '停权_x', '锁定_x', '序_y', '会员帐号_y', '真实姓名_y', '直属好友_y', '代理帐号_y', '注册时间_y',
       '注册国家_y', '注册城市_y', '主帐户余额_y', '损益_y', '会员层级_y', '启/停用_y', '冻结_y',
       '停权_y', '锁定_y'],
      dtype='object')

In [18]:
final_user=test_user[test_user['会员帐号_y'].notnull()][['会员帐号_x','注册时间_x','注册IP','会员帐号_y','注册时间_y']]
final_user.columns = ['会员帐号_YABO', '注册时间_YABO', '注册IP','会员帐号_SG','注册时间_SG']

In [19]:
final_user.to_csv('F:/Desktop/newuser_0113.csv' ,encoding="utf_8_sig" )